In [1]:
!pip install pip install openai==0.28 langchain faiss-cpu tiktoken

In [ ]:
from google.colab import userdata
userdata.get('OPENAI_API_KEY')

In [3]:
!pip install requests scrapy selenium

In [6]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 libudev1 snapd squashfs-tools systemd-hwe-hwdb
  udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver libfuse3-3 liblzo2-2 snapd squashfs-tools
  systemd-hwe-hwdb udev
The following packages will be upgraded:
  libudev1
1 upgraded, 9 newly installed, 0 to remove and 44 not upgraded.
Need to get 27.3 MB of archives.
After this operation, 114 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 apparmor amd64 3.0.4-2ubuntu2.3 [595 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblzo2-2 amd64 2.10-2build3 [53.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 squashfs-tools amd64 1:4.5-3

In [7]:
import numpy as np
import pandas as pd
from scrapy.selector import Selector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [8]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

In [9]:
url = "https://www.imdb.com/title/tt1517268/reviews/?ref_=tt_ov_rt"
driver.get(url)

In [10]:
sel = Selector(text = driver.page_source)
review_counts = sel.css('.lister .header span::text').extract_first().replace(',','').split(' ')[0]
more_review_pages = int(int(review_counts)/25)

In [11]:
for i in tqdm(range(more_review_pages)):
    try:
        css_selector = 'load-more-trigger'
        driver.find_element(By.ID, css_selector).click()
    except:
        pass

100%|██████████| 70/70 [00:03<00:00, 22.83it/s]


In [12]:
rating_list = []
review_date_list = []
review_title_list = []
author_list = []
review_list = []
review_url_list = []
error_url_list = []
error_msg_list = []
reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review-container')

for d in tqdm(reviews):
    try:
        sel2 = Selector(text = d.get_attribute('innerHTML'))
        try:
            rating = sel2.css('.rating-other-user-rating span::text').extract_first()
        except:
            rating = np.NaN
        try:
            review = sel2.css('.text.show-more__control::text').extract_first()
        except:
            review = np.NaN
        try:
            review_date = sel2.css('.review-date::text').extract_first()
        except:
            review_date = np.NaN
        try:
            author = sel2.css('.display-name-link a::text').extract_first()
        except:
            author = np.NaN
        try:
            review_title = sel2.css('a.title::text').extract_first()
        except:
            review_title = np.NaN
        try:
            review_url = sel2.css('a.title::attr(href)').extract_first()
        except:
            review_url = np.NaN
        rating_list.append(rating)
        review_date_list.append(review_date)
        review_title_list.append(review_title)
        author_list.append(author)
        review_list.append(review)
        review_url_list.append(review_url)
    except Exception as e:
        error_url_list.append(url)
        error_msg_list.append(e)
review_df = pd.DataFrame({
    'Review_Date':review_date_list,
    'Author':author_list,
    'Rating':rating_list,
    'Review_Title':review_title_list,
    'Review':review_list,
    'Review_Url':review_url
    })

100%|██████████| 100/100 [00:00<00:00, 121.64it/s]


In [13]:
review_df

,Review_Date,Author,Rating,Review_Title,Review,Review_Url
0,21 July 2023,LoveofLegacy,6,"Beautiful film, but so preachy\n","Margot does the best with what she's given, bu...",/review/rw9200246/?ref_=tt_urv
1,26 July 2023,aherdofbeautifulwildponies,6,A Hot Pink Mess\n,"Before making Barbie (2023),",/review/rw9200246/?ref_=tt_urv
2,23 July 2023,Revuer223,6,Could Have Been Great. 2nd Half Brings It Dow...,"The quality, the humor, and the writing of the...",/review/rw9200246/?ref_=tt_urv
3,21 July 2023,Sleepin_Dragon,8,Well this really did come as a surprise.\n,"It pains me to say it, but I enjoyed this movi...",/review/rw9200246/?ref_=tt_urv
4,22 July 2023,Natcat87,6,Too heavy handed\n,"As a woman that grew up with Barbie, I was ver...",/review/rw9200246/?ref_=tt_urv
...,...,...,...,...,...,...
95,22 July 2023,fernandoschiavi,5,"""Barbie"" is fun and visually beautiful, but u...","Despite the strong commercial tone, it is stil...",/review/rw9200246/?ref_=tt_urv
96,3 August 2023,anonim-73883,10,Great philosophical film to think about and l...,This is.... Was.... Admirably!,/review/rw9200246/?ref_=tt_urv
97,10 September 2023,Lincsobserver,6,"Colourful, but confusing\n",Who exactly was this film aimed at? Most of th...,/review/rw9200246/?ref_=tt_urv
98,25 July 2023,kimberlyely-27550,10,Everyone watch this\n,Barbie,/review/rw9200246/?ref_=tt_urv


In [15]:
review_df.to_csv('/content/reviews_csv/barbie.csv')

### Load Data

In [4]:
from langchain.document_loaders.csv_loader import CSVLoader

In [5]:
loader = CSVLoader(
    file_path = '/content/reviews_csv/barbie.csv',
    source_column = 'Review_Url'
)

data = loader.load()

In [6]:
print(data)

[Document(page_content=': 0\nReview_Date: 21 July 2023\nAuthor: LoveofLegacy\nRating: 6\nReview_Title: Beautiful film, but so preachy\nReview: Margot does the best with what she\'s given, but this film was very disappointing to me. It was marketed as a fun, quirky satire with homages to other movies. It started that way, but ended with over-dramatized speeches and an ending that clearly tried to make the audience feel something, but left everyone just feeling confused. And before you say I\'m a crotchety old man, I\'m a woman in my 20s, so I\'m pretty sure I\'m this movie\'s target audience. The saddest part is there were parents with their kids in the theater that were victims of the poor marketing, because this is not a kid\'s movie. Overall, the humor was fun on occasion and the film is beautiful to look at, but the whole concept falls apart in the second half of the film and becomes a pity party for the "strong" woman.\nReview_Url: /review/rw9200246/?ref_=tt_urv', metadata={'source

In [7]:
len(data)

100

### Split data

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100,
    length_function = len,
)

documents = splitter.transform_documents(data)

In [9]:
print(documents)

[Document(page_content=': 0\nReview_Date: 21 July 2023\nAuthor: LoveofLegacy\nRating: 6\nReview_Title: Beautiful film, but so preachy\nReview: Margot does the best with what she\'s given, but this film was very disappointing to me. It was marketed as a fun, quirky satire with homages to other movies. It started that way, but ended with over-dramatized speeches and an ending that clearly tried to make the audience feel something, but left everyone just feeling confused. And before you say I\'m a crotchety old man, I\'m a woman in my 20s, so I\'m pretty sure I\'m this movie\'s target audience. The saddest part is there were parents with their kids in the theater that were victims of the poor marketing, because this is not a kid\'s movie. Overall, the humor was fun on occasion and the film is beautiful to look at, but the whole concept falls apart in the second half of the film and becomes a pity party for the "strong" woman.\nReview_Url: /review/rw9200246/?ref_=tt_urv', metadata={'source

In [10]:
len(documents)

151

### Embedding + Index

In [11]:
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

os.environ['OPENAI_API_KEY'] = "Enter Your OPENAI_API_KEY"
embeddings = OpenAIEmbeddings()

vectordb = FAISS.from_documents(
    documents,
    embeddings
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


### Query

In [12]:
query = "How is Will Ferrell in this movie?"
embedding_vector = embeddings.embed_query(query)
docs = vectordb.similarity_search_by_vector(
    embedding_vector,
    k = 3
)

for page in docs:
  print(page.page_content)

I can imagine that in the boardroom they went through a discussion like this: People are going to think this is a feel good movie so we need to give them something deeper but also lets make it absurd and fast paced so that we can finish the plot quickly and also jump on the "not taking itself serious" bandwagon and also market these dolls again in the age of smartphone kids. The only reason this movie gets at 5 from me is the humor, which sometimes becomes very meta and creative and Margot Robbie and Ryan Gosling have acted very well. Besides this, they could have just given us a simple feel good movie and I would have respected the creators more. Pass.
Review_Url: /review/rw9200246/?ref_=tt_urv
While i'm not so sure at first, the movie kept getting even more fun, entertaining, and definitely better, also surprisingly deal with a legit serious stuff, Barbie is a weirdly fun movie that fills with this very interesting concept, definitely the first time that's ever done, Greta Gerwig has

### Retrieval Chain

In [13]:
from langchain.llms.openai import OpenAIChat

llm = OpenAIChat(
    model = "gpt-3.5-turbo"
)

retriever = vectordb.as_retriever()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIChat` was deprecated in LangChain 0.0.1 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_community/llms/openai.py:1070: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


In [14]:
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler

handler = StdOutCallbackHandler()

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = retriever,
    callbacks = [handler],
    return_source_documents = True
)

In [15]:
qa_chain(
    {
        "query" : "How was Will Ferrell in this movie?"
    }
)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'How was Will Ferrell in this movie?',
 'result': 'Will Ferrell was great and entertaining according to one of the reviews.',
 'source_documents': [Document(page_content="Review: The movie was very funny and really enjoyable to laugh at with the full theatre. However, the messages of the movie were the problem. I was never really sure what I was supposed to take away, there was nothing about finding equality or love it was all about how every man cat falls every woman or women can't be anything. It was really silly because there was no accurate reflection of America at any single point except for Barbie getting called a Fascist for no reason by a 14 year old. I enjoyed how they called out women for hating women and how they really tried to preach empowerment and the ability to be anything, but at the same time there was so much resentment and they ended the movie by reinstating hate. The majority of the movie was hating men as much as possible. That's just whatever because wh

In [18]:
results = qa_chain(
    {
        "query" : "Do reviewers consider this movie Kenough?"
    }
)
results['result']



> Entering new RetrievalQA chain...

> Finished chain.


'Yes, the reviewers have mixed opinions on whether the movie is Kenough.'